## Import Libraries

In [ ]:
# import libraries

import great_expectations as ge
from great_expectations.data_context import FileDataContext
import pandas as pd
import numpy as np

In [65]:
# create data context

project_root_dir = './'
context = FileDataContext.create(project_root_dir=project_root_dir)

datasource_name = 'hotel_bookings_data'
datasource = context.sources.add_pandas(datasource_name)

In [66]:
def add_unique_column(path_to_data):
    data = pd.read_csv(path_to_data)
    # create a unique identifier for each row that represents booking based on record_number in each hotel_type and arrival_date
    data['uid'] = data['hotel_type'] + "_" + data['arrival_date'] + data['record_number'].astype(str)
    modified_data_path = f'{project_root_dir}/P2M3_galuh_alifani_data_clean_with_uid.csv'
    data.to_csv(modified_data_path, index=False)
    return modified_data_path

In [ ]:
asset_name = 'hotel_bookings_asset_gx'
path_to_data = './dags/data_clean.csv'
modified_data_path = add_unique_column(path_to_data)
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=modified_data_path)

# create batch request
batch_request = asset.build_batch_request()

## Expectations Suite Setup

In [68]:
expectation_suite_name = 'expectation-hotel-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reservation_status_date,hotel_type,record_number,arrival_date,is_family,total_stay_nights,total_rate,is_room_type_changed,total_guests,uid
0,0,7,2015,7,27,1,0,1,1,0,...,2015-07-02,resort_hotel,3,2015-07-01,0,1,75.0,1,1,resort_hotel_2015-07-013
1,0,13,2015,7,27,1,0,1,1,0,...,2015-07-02,resort_hotel,4,2015-07-01,0,1,75.0,0,1,resort_hotel_2015-07-014
2,0,14,2015,7,27,1,0,2,2,0,...,2015-07-03,resort_hotel,5,2015-07-01,0,2,196.0,0,2,resort_hotel_2015-07-015
3,0,14,2015,7,27,1,0,2,2,0,...,2015-07-03,resort_hotel,6,2015-07-01,0,2,196.0,0,2,resort_hotel_2015-07-016
4,0,0,2015,7,27,1,0,2,2,0,...,2015-07-03,resort_hotel,7,2015-07-01,0,2,214.0,0,2,resort_hotel_2015-07-017


## Expectations

### 1. to be unique
Column `uid` (consist of booking_id for each of the hotels) must be unique

In [69]:
validator.expect_column_values_to_be_unique('uid')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 117481,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {}
}

### 2. to be between min_value and max_value
Total stay duration per booking `total_stay_nights` must be between 0 - 90 days (3 months) --> 0 days are for canceled & no show bookings

In [70]:
validator.expect_column_values_to_be_between(
    column='total_stay_nights', min_value=0, max_value=90
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 117481,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {}
}

### 3. to be in set
Value of `reservation_status` must be either ['Check-Out', 'Canceled', 'No-Show']

In [71]:
validator.expect_column_values_to_be_in_set('reservation_status', ['Check-Out', 'Canceled', 'No-Show'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 117481,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {}
}

### 4. to be in type list
Value of `adr` must be either ['integer', 'float']

In [72]:
validator.expect_column_values_to_be_in_type_list('adr', ['integer', 'float'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": "float64"
  },
  "meta": {}
}

### 5. to be of type
Value of `total_guests` must be an integer

In [73]:
validator.expect_column_values_to_be_of_type('total_guests', 'int')

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": "int64"
  },
  "meta": {}
}

### 6. to match regex
Value of `country` must only contain valid alphabets

In [74]:
validator.expect_column_values_to_match_regex('country', r"^[A-Z]$")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 117481,
    "unexpected_count": 117481,
    "unexpected_percent": 100.0,
    "partial_unexpected_list": [
      "GBR",
      "GBR",
      "GBR",
      "GBR",
      "PRT",
      "PRT",
      "PRT",
      "PRT",
      "PRT",
      "PRT",
      "USA",
      "ESP",
      "PRT",
      "IRL",
      "PRT",
      "IRL",
      "FRA",
      "GBR",
      "GBR",
      "PRT"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 100.0,
    "unexpected_percent_nonmissing": 100.0
  },
  "meta": {}
}

### 7. to equal length
Character length of `meal` column must be 2

In [75]:
validator.expect_column_value_lengths_to_equal('meal', 2)


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 117481,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {}
}